# MCPricing Library Walkthrough

This notebook documents the library in five parts:
1. Introduction of capabilities (implemented in this version)
2. Mathematical techniques
3. Library architecture
4. Comparison tests across valuation techniques
5. How to build your own payoff

## 1) Introduction of capabilities

The current library supports:
- Equity payoffs: European, Asian, Digital, Barrier, American, Bermudan, Autocall
- Pricing styles: closed-form (where available), Monte Carlo, binomial tree (Jarrow-Rudd), Dupire local-vol Monte Carlo
- Structured products through payoff composition and path-dependent `cash_flows`

Core design idea:
- Instruments define the pricing interface
- Payoffs define how path observations become cashflows
- Simulators generate market paths (BS, tree, Dupire)

In [7]:
from EquityInstruments import (
    EuropeanCall, AsianCallOption, DigitalCallOption,
    BarrierCallOption, AmericanCallOption, BermudanCallOption,
    AutocallPutOption
)

spot = 100
strike = 100
r = 0.02
q = 0.01
sigma = 0.2
T = 1.0

euro = EuropeanCall(spot, strike, r, q, sigma, T)
asian = AsianCallOption(spot, strike, r, q, sigma, [0.25, 0.5, 0.75, 1.0], T)
digital = DigitalCallOption(spot, strike, r, q, sigma, T)
barrier = BarrierCallOption(spot, strike, 120, r, q, sigma, T, u_d='u', activation=True)
american = AmericanCallOption(spot, strike, r, q, sigma, T)
bermudan = BermudanCallOption(spot, strike, r, q, sigma, T, [0.5, 0.75])

dates = [0.25, 0.5, 0.75, 1.0]
autocall = AutocallPutOption(
    notional=100, S0=spot, r=r, q=q,
    dates=dates, dates_obs=dates,
    coupon=0.02, coup_barrier=0.7, autocall_barrier=1.0,
    put_strike=0.8, isAcumulative=True, isLeveraged=False
)

products = [
    ("European", euro),
    ("Asian", asian),
    ("Digital", digital),
    ("Barrier", barrier),
    ("American", american),
    ("Bermudan", bermudan),
    ("Autocall", autocall),
]

for name, p in products:
    print(f"{name:10} -> models: {list(p.models.keys())}")

European   -> models: ['BS', 'JarrowRuddTree', 'Dupire']
Asian      -> models: ['BS', 'Dupire']
Digital    -> models: ['BS', 'Dupire']
Barrier    -> models: ['BS', 'Dupire']
American   -> models: ['JarrowRuddTree']
Bermudan   -> models: ['JarrowRuddTree']
Autocall   -> models: ['BS', 'Dupire']


### Autocall and structured product valuation

Autocall payoff is path-dependent and cashflow-based:
- At each observation date, coupon barriers are checked
- Coupons can accumulate (if configured)
- If autocall barrier is hit after allowed start period, product redeems early
- If never autocalled, maturity redemption and maturity-option adjustment are applied

This behavior is implemented in `AutocallMultiCouponPayoff.cash_flows(...)`.

In [8]:
import numpy as np
from Payoffs import AutocallMultiCouponPayoff, EuropeanPutPayOff

# Example payoff definition
notional = 100
coupons = [0.02]                   # 2% coupon
coupon_barriers = [0.70]           # coupon paid if spot >= 70% of initial
autocall_barrier = 1.00            # early redemption if spot >= 100%
is_accumulative = True
autocall_start_period = 2          # from second observation onward

# Maturity adjustment: short one put-style leg
maturity_payoffs = [EuropeanPutPayOff(0.80)]
maturity_parts = [1.0]

payoff = AutocallMultiCouponPayoff(
    notional, coupons, coupon_barriers, autocall_barrier,
    is_accumulative, maturity_payoffs, maturity_parts,
    autocall_start_period=autocall_start_period
)

# Toy path in normalized spot terms S_t / S_0
obs = np.array([0.85, 1.03, 0.95, 0.90])
discounts = np.array([0.995, 0.990, 0.985, 0.980])

cf = payoff.cash_flows(obs)
pv = np.dot(cf, discounts)

print("Observations:", obs)
print("Cashflows:", cf)
print("PV:", pv)

Observations: [0.85 1.03 0.95 0.9 ]
Cashflows: [  2. 102.   0.   0.]
PV: 102.97


### Generic structured product valuation with a simple custom payoff

Any payoff can be priced as long as it exposes `cash_flows(observations)` (or `value(...)`).
Below is a simple custom payoff with two cashflow dates.

In [9]:
from Payoffs import Payoff

class SimpleStructuredPayoff(Payoff):
    def __init__(self, notional, trigger=1.0):
        self.notional = notional
        self.trigger = trigger

    def value(self, observations, **kwargs):
        # Not used directly in this example
        return self.cash_flows(observations)[-1]

    def cash_flows(self, observations):
        # Two-date toy structure
        # Date 1: pay 3% coupon if first observation > trigger
        # Date 2: always return notional
        cf = np.zeros(len(observations))
        if observations[0] > self.trigger:
            cf[0] = 0.03 * self.notional
        cf[-1] += self.notional
        return cf

toy_payoff = SimpleStructuredPayoff(notional=100, trigger=1.0)
toy_obs = np.array([1.05, 0.97])
toy_disc = np.array([0.99, 0.97])

print("Toy cashflows:", toy_payoff.cash_flows(toy_obs))
print("Toy PV:", np.dot(toy_payoff.cash_flows(toy_obs), toy_disc))

Toy cashflows: [  3. 100.]
Toy PV: 99.97


## 2) Mathematical techniques

This section summarizes the core quantitative methods used in the library:
- Monte Carlo pricing
- Tree pricing (constant volatility)
- Dupire local-vol calibration


### 2.1 Monte Carlo pricing

Under risk-neutral dynamics, price is the discounted expectation of payoff cashflows:

$ V_0 = \mathbb{E}^\mathbb{Q}[\sum_i D(0,t_i) \cdot CF_i] $

In this library:
- The simulator generates paths at observation dates
- The payoff maps observations to `cash_flows`
- The instrument applies discount factors and averages across simulations


In [10]:
from EquityInstruments import EuropeanCall

spot = 100
strike = 100
r = 0.02
q = 0.01
sigma = 0.2
T = 1.0

op = EuropeanCall(spot, strike, r, q, sigma, T)

# Closed-form Black-Scholes vs Monte Carlo under constant vol
price_bs = op.price(model='BS')
price_mc = op.price(model='BS', simulation=True, n_sims=50000)

print('BS closed form :', round(price_bs, 6))
print('BS Monte Carlo :', round(price_mc, 6))
print('Difference     :', round(price_mc - price_bs, 6))


BS closed form : 8.349406
BS Monte Carlo : 8.36316
Difference     : 0.013754


### 2.2 Tree pricing (constant volatility)

For early-exercise products (American/Bermudan), the library uses a Jarrow-Rudd binomial tree.
The algorithm is backward induction:
1. Set terminal node values with payoff at maturity
2. Roll back node by node under risk-neutral probabilities
3. At decision dates, take max(intrinsic, continuation)


In [11]:
from EquityInstruments import AmericanPutOption, EuropeanPutOption

spot = 100
strike = 100
r = 0.03
q = 0.00
sigma = 0.25
T = 1.0

eu_put = EuropeanPutOption(spot, strike, r, q, sigma, T)
am_put = AmericanPutOption(spot, strike, r, q, sigma, T)

price_eu_bs = eu_put.price(model='BS')
price_am_tree = am_put.price(model='JarrowRuddTree', deltat=1/252)

print('European Put (BS)      :', round(price_eu_bs, 6))
print('American Put (Tree)    :', round(price_am_tree, 6))
print('Early exercise premium :', round(price_am_tree - price_eu_bs, 6))


European Put (BS)      : 8.39303
American Put (Tree)    : 8.672013
Early exercise premium : 0.278983


### 2.3 Dupire local-vol calibration

Dupire pricing in this library follows:
1. Build a maturity-strike grid
2. Provide market surface input (implied vols or prices)
3. Calibrate local volatility surface
4. Run Monte Carlo with local vol and curve-consistent discounting

The `Dupire` model entrypoint in equity instruments does this calibration + pricing workflow.


In [25]:
import numpy as np
from Utils import VolatilityMatrix
from EquityInstruments import EuropeanCall

tenors = np.array([0.25, 0.50, 1.00, 2.00])
strikes = np.array([80, 90, 100, 110, 120])
tenors = np.array([0.25, 0.50, 1.00, 2.00])
strikes = np.array([80, 90, 100, 110, 120])

T, K = np.meshgrid(tenors, strikes, indexing="ij")

iv_surface = (
    0.18
    + 0.015 * np.sqrt(T)
    - 0.0008 * (K - 100)
    + 0.00002 * (K - 100) ** 2
)

iv_surface = np.maximum(iv_surface, 0.01)

op = EuropeanCall(100, 100, 0.02, 0.01, 0.20, 1.0)

price_dupire = op.price(
    model='Dupire',
    n_sims=100000,
    vol_matrix=iv_surface,
    strikes=strikes,
    tenors=tenors,
    vol_floor=1e-8
)

iv_interp = VolatilityMatrix(tenors, strikes, iv_surface)
sigma_tk = float(iv_interp(1.0, 100.0))  # IV interpolada en (T,K)

price_bs_iv = op.price(model='BS', sigma=sigma_tk)

print("Sigma interp (T=1,K=100):", sigma_tk)
print("European Call (BS with IV surface):", round(price_bs_iv, 6))
print("European Call (Dupire MC):        ", round(price_dupire, 6))


Sigma interp (T=1,K=100): 0.195
European Call (BS with IV surface): 8.154111
European Call (Dupire MC):         7.871741


## 3) Library architecture

This section maps how modules collaborate and where each responsibility lives.

### 3.1 Module responsibilities

- `Payoffs.py`
  - Defines payoff logic (`value`, `cash_flows`, early-exercise decisions).
  - Path-dependent structures (e.g., autocall) are implemented here.

- `Instruments.py`
  - Defines product pricing interfaces (`price`, `priceMC`, `priceTree`).
  - Connects payoff + simulator + discounting into a final PV.
  - Includes shared Dupire calibration helpers in `OneStrikeOption`.

- `EquityInstruments.py`
  - Concrete equity products (European, Asian, Digital, Barrier, American, Bermudan, Autocall).
  - Model routing (`BS`, `JarrowRuddTree`, `Dupire`).

- `SimulatorEngine.py`
  - Stochastic path generators (Black-Scholes, tree, Dupire local vol).
  - Dupire surface calibration and simulation dynamics.

- `Utils.py`
  - Shared numerical utilities (2D interpolation matrices, local-vol wrapper, discount from zero curve).

### 3.2 Pricing flow (high level)

`Instrument.price(model=...)`
-> selects model-specific method in the product class
-> builds/uses simulator (BS, Tree, Dupire)
-> asks payoff for `cash_flows(path_observations)`
-> discounts cashflows
-> aggregates (expectation/backward induction)
-> returns present value

### 3.3 Class interaction map

`EquityOption` (base pricing interface)
-> `OneStrikeOption` (shared market fields + Dupire helper workflow)
-> concrete products in `EquityInstruments.py`

`Payoff`
-> concrete payoff classes in `Payoffs.py`

`Simulator` / `TreeSimulator`
-> concrete engines in `SimulatorEngine.py`

Coupling rule: products orchestrate, payoffs define economics, simulators define dynamics.

In [13]:
# Quick architecture introspection from live objects
from EquityInstruments import EuropeanCall, AutocallPutOption

ec = EuropeanCall(100, 100, 0.02, 0.01, 0.2, 1.0)
ac = AutocallPutOption(
    100, 100, 0.02, 0.01,
    [0.25, 0.5, 0.75, 1.0], [0.25, 0.5, 0.75, 1.0],
    0.02, 0.7, 1.0, 0.8, True, False
)

print('EuropeanCall class        :', ec.__class__.__name__)
print('EuropeanCall payoff class :', ec.payoff_.__class__.__name__)
print('EuropeanCall models       :', list(ec.models.keys()))
print()
print('Autocall class            :', ac.__class__.__name__)
print('Autocall payoff class     :', ac.payoff_.__class__.__name__)
print('Autocall models           :', list(ac.models.keys()))


EuropeanCall class        : EuropeanCall
EuropeanCall payoff class : EuropeanCallPayOff
EuropeanCall models       : ['BS', 'JarrowRuddTree', 'Dupire']

Autocall class            : AutocallPutOption
Autocall payoff class     : AutocallMultiCouponPayoff
Autocall models           : ['BS', 'Dupire']


### 3.4 Why this architecture works

- Extensibility: new products can reuse existing simulators and payoff primitives.
- Testability: payoff logic, simulators, and pricing wrappers can be tested independently.
- Model portability: same product can expose multiple models (`BS`, `Tree`, `Dupire`) with one interface.

## 4) Test comparing valuation techniques

This section provides reproducible comparisons between valuation approaches.

Targets:
1. Monte Carlo vs Tree under constant volatility
2. Constant-vol Monte Carlo vs Dupire Monte Carlo


### 4.1 MC vs Tree (constant volatility)

We compare a European vanilla option priced with:
- Monte Carlo under Black-Scholes dynamics
- Jarrow-Rudd binomial tree
- Closed-form BS reference


In [14]:
import numpy as np
from EquityInstruments import EuropeanCall

np.random.seed(12345)

spot = 100
strike = 100
r = 0.02
q = 0.01
sigma = 0.20
T = 1.0

op = EuropeanCall(spot, strike, r, q, sigma, T)

price_bs_ref = op.price(model='BS')
price_mc_const = op.price(model='BS', simulation=True, n_sims=100000)
price_tree_const = op.price(model='JarrowRuddTree', deltat=1/252)

print('European Call (BS closed form):', round(price_bs_ref, 6))
print('European Call (MC constant vol):', round(price_mc_const, 6))
print('European Call (Tree constant) :', round(price_tree_const, 6))
print('MC - BS  :', round(price_mc_const - price_bs_ref, 6))
print('Tree - BS:', round(price_tree_const - price_bs_ref, 6))


European Call (BS closed form): 8.349406
European Call (MC constant vol): 8.351078
European Call (Tree constant) : 8.356369
MC - BS  : 0.001673
Tree - BS: 0.006963


### 4.2 MC constant vol vs Dupire MC

Now we keep the product fixed and compare:
- Constant-vol MC (`BS` simulation mode)
- Dupire MC after local-vol calibration from an implied-vol surface

For a flat IV surface, both methods should be close (up to MC noise and calibration numerics).

In [15]:
import numpy as np
from EquityInstruments import EuropeanCall
from Utils import VolatilityMatrix

np.random.seed(2026)

spot = 100
strike = 100
r = 0.02
q = 0.01
sigma_const = 0.22
T = 1.0

tenors = np.array([0.25, 0.50, 1.00, 2.00])
strikes = np.array([80, 90, 100, 110, 120])
iv_surface = np.full((len(tenors), len(strikes)), sigma_const)

op = EuropeanCall(spot, strike, r, q, sigma_const, T)

price_mc_const = op.price(model='BS', simulation=True, n_sims=100000)

price_dupire = op.price(
    model='Dupire',
    n_sims=100000,
    vol_matrix=iv_surface,
    strikes=strikes,
    tenors=tenors,
    vol_floor=1e-8
)

iv_interp = VolatilityMatrix(tenors, strikes, iv_surface)
sigma_tk = float(iv_interp(T, strike))
price_bs_iv = op.price(model='BS', sigma=sigma_tk)

print('Sigma(T,K) from surface  :', round(sigma_tk, 6))
print('European Call (BS with IV):', round(price_bs_iv, 6))
print('European Call (MC const) :', round(price_mc_const, 6))
print('European Call (Dupire MC):', round(price_dupire, 6))
print('Dupire - MC const        :', round(price_dupire - price_mc_const, 6))


Sigma(T,K) from surface  : 0.22
European Call (BS with IV): 9.130199
European Call (MC const) : 9.159326
European Call (Dupire MC): 9.156627
Dupire - MC const        : -0.002699


## 5) How to make your own payoff

This section shows the minimum interface and a full custom-payoff pricing example.

### 5.1 Minimum contract for a payoff

To be compatible with the pricing engines, your custom class should inherit from `Payoff` and implement:
- `value(observations, **kwargs)`

If your product has intermediate cashflows, also implement:
- `cash_flows(observations)`

Optionally for early exercise (tree methods):
- `hasDecision(date, deltat)`


In [17]:
from Payoffs import Payoff
import numpy as np

class CliquetLikePayoff(Payoff):
    """
    Toy path-dependent payoff with local cap/floor on period returns.
    - observations are normalized levels (S_t / S_0) on monitoring dates
    """
    def __init__(self, notional, local_floor=-0.05, local_cap=0.05, global_floor=0.0):
        self.notional = float(notional)
        self.local_floor = float(local_floor)
        self.local_cap = float(local_cap)
        self.global_floor = float(global_floor)

    def value(self, observations, **kwargs):
        obs = np.asarray(observations, dtype=float)
        if obs.ndim != 1 or obs.size < 2:
            raise ValueError('CliquetLikePayoff needs at least 2 observations')

        period_ret = obs[1:] / obs[:-1] - 1.0
        clipped = np.clip(period_ret, self.local_floor, self.local_cap)
        total = max(clipped.sum(), self.global_floor)
        return self.notional * total


### 5.2 Price a custom payoff with an existing instrument

You can reuse `EquityOption.priceMC(...)` from any equity instrument class as a pricing wrapper.
Here we use `EuropeanCall` only as a host for market inputs and discounting.

In [18]:
from EquityInstruments import EuropeanCall
from SimulatorEngine import BlackScholesSimulator

spot = 100
r = 0.02
q = 0.01
sigma = 0.20
T = 1.0
dates = [0.25, 0.50, 0.75, 1.00]

# Host instrument: we only use shared MC pricing + discount interface
host = EuropeanCall(spot, 100, r, q, sigma, T)
sim = BlackScholesSimulator(spot, r, q, sigma)
payoff = CliquetLikePayoff(notional=100, local_floor=-0.03, local_cap=0.04, global_floor=0.0)

price_custom = host.priceMC(sim, payoff, dates=dates, n_sims=50000)
print('Custom payoff MC price:', round(price_custom, 6))


Custom payoff MC price: 2.679598


### 5.3 Path with intermediate coupons: implement `cash_flows`

If the payoff has intermediate payments, return a cashflow vector aligned with pricing dates.
The engine applies discounting date by date and then aggregates the PV.

In [19]:
class CouponAndRedemptionPayoff(Payoff):
    def __init__(self, notional, coupon_rate, trigger=1.0):
        self.notional = float(notional)
        self.coupon_rate = float(coupon_rate)
        self.trigger = float(trigger)

    def value(self, observations, **kwargs):
        # Fallback (not used by MC when cash_flows is provided)
        return self.cash_flows(observations)[-1]

    def cash_flows(self, observations):
        obs = np.asarray(observations, dtype=float)
        cf = np.zeros(obs.size)
        # coupon each date if level above trigger
        cf += (obs > self.trigger) * (self.notional * self.coupon_rate)
        # final redemption
        cf[-1] += self.notional
        return cf

payoff2 = CouponAndRedemptionPayoff(notional=100, coupon_rate=0.01, trigger=0.95)
price_custom_cf = host.priceMC(sim, payoff2, dates=dates, n_sims=50000)
print('Custom coupon+redemption MC price:', round(price_custom_cf, 6))


Custom coupon+redemption MC price: 101.97024


### 5.4 Checklist for robust custom payoffs

- Validate shape/type of `observations`
- Keep payoff units consistent (normalized spots vs absolute levels)
- If using `cash_flows`, ensure vector length matches pricing dates
- For tree early exercise, implement `hasDecision` explicitly
- Add deterministic toy-path tests before running large MC